# Artificial Neural Network (ANN)

Bank dataset:
The company has been seeing unusual churn rates, customers are leaving the company and there are high rates 
- don't have missing data


### Importing the libraries

In [83]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [84]:
tf.__version__

'2.12.0'

# Part 1 - Data Preprocessing

### Importing the dataset

In [85]:
dataset = pd.read_csv('Churn_Modelling.csv')

# not including the row number, customer ID and surname
# all the values except the last column
x = dataset.iloc[:, 3:-1].values
# just the last column (output)
y = dataset.iloc[:, -1].values

print("x -> ", x)
print("y -> ", y)

x ->  [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y ->  [1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [86]:
from sklearn.preprocessing import LabelEncoder

# for the gender column
le = LabelEncoder()

gender_column = x[:, 2]
print(gender_column)

x[:, 2] = le.fit_transform(x[:, 2])

gender_column = x[:, 2]
print(gender_column)


['Female' 'Female' 'Female' ... 'Female' 'Male' 'Female']
[0 0 0 ... 0 1 0]


One Hot Encoding the "Geography" column

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# for the geography column
geography_column = x[:, 1]
print(geography_column)

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

geography_column = x[:, 1]
print(geography_column)


['France' 'Spain' 'France' ... 'France' 'Germany' 'France']
[0.0 0.0 0.0 ... 0.0 1.0 0.0]
['France' 'Spain' 'France' ... 'France' 'Germany' 'France']
[0.0 0.0 0.0 ... 0.0 1.0 0.0]


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [89]:
# feature scaling is very important for deep learning 
# is more important for deep learning due to the multiple layers and numerous parameters
# ensures that all input features contribute equally to the model

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)
print(x_train.shape)

x_test = sc.transform(x_test)

(8000, 12)


# Part 2 - Building the ANN

### Initializing the ANN

In [90]:
# initialises the ann as sequence of layers
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [91]:
"""
    units = number of neurons in the hidden layer 
    there is no rule to choose the number of neurons, its all about experimentation
    
    activation = the activation function we will use = rectifier function
"""

# all the neurons will have the same activation function
hidden_layer_1 = tf.keras.layers.Dense(6, activation='relu')

ann.add(hidden_layer_1)

### Adding the second hidden layer

In [92]:
hidden_layer_2 = tf.keras.layers.Dense(6, activation='relu')

ann.add(hidden_layer_2)

### Adding the output layer

In [93]:
# sigmoid - give us the probability that the customer leaves the bank (prob the output to be 1)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

ann.add(output_layer)

# Part 3 - Training the ANN

### Compiling the ANN

In [94]:
ann.compile(
    optimizer ='adam',
    loss= 'binary_crossentropy',
    metrics = ['accuracy']
)

### Training the ANN on the Training set

In [95]:
ann.fit(
    x_train, 
    y_train, 
    batch_size = 32, 
    epochs = 100
)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6570 - accuracy: 0.6321
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4960 - accuracy: 0.7975
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4570 - accuracy: 0.8031
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4379 - accuracy: 0.8104
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4285 - accuracy: 0.8134
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4221 - accuracy: 0.8152
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4169 - accuracy: 0.8192
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4127 - accuracy: 0.8210
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4093 - accuracy: 0.8223
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4054 - accura

# Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [96]:
customer = [[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]
scaled = sc.transform(customer)

ann_result = ann.predict(scaled)
print(ann_result)

prediction = (ann_result > 0.5)

# Output the result
if prediction:
    print("Yes, we should say goodbye to that customer.")
else:
    print("No, the customer is likely to stay.")

1/1 [==============================] - 0s 57ms/step
[[0.02868822]]
No, the customer is likely to stay.


**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [97]:
y_pred = ann.predict(x_test)

y_pred = (y_pred > 0.5)

y_pred_reshapred = y_pred.reshape(len(y_pred), 1)
y_test_reshaped = y_test.reshape(len(y_test), 1)

concatenated = np.concatenate((y_pred_reshapred, y_test_reshaped), 1)

print(concatenated)

63/63 [==============================] - 0s 716us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [98]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1509   86]
 [ 199  206]]


0.8575